In [79]:
from osgeo import gdal
import pandas as pd
import os
import rasterio
from rasterio.merge import merge 
import numpy as np
import glob
import matplotlib.pyplot as plt

In [80]:
def updateGeoTransforms(srcGeo, xOff, yOff):
    
    out = [srcGeo[0] + xOff * srcGeo[1], srcGeo[1], srcGeo[2], srcGeo[3] + yOff * srcGeo[5], srcGeo[4], srcGeo[5]]
    return out

def makeFileName(x, suffix, ext = "tif"):
    
    base = os.path.splitext(os.path.basename(x))[0]
    return base + '.' + suffix + '.' + ext

In [81]:
def hdf_to_correcttif(hdf_file,tif_dir):
    sds = gdal.Open(hdf_file, gdal.GA_ReadOnly).GetSubDatasets()
    vi = gdal.Open(sds[0][0])
    width = 4800
    height= 4800
    xOff = 0
    yOff = 0
    vi_src = gdal.Open(sds[0][0])
    vi_np = vi_src.ReadAsArray(xOff, yOff, width, height)
    
    QA_src = gdal.Open(sds[11][0])
    QA_np = QA_src.ReadAsArray(xOff, yOff, width, height)
    
    vi_np[np.logical_and(QA_np != 0, QA_np != 1)] = -3000
    QA_np = None
    
    geoT = vi.GetGeoTransform()
    proj = vi.GetProjection()
    
    ofile=tif_dir+hdf_file.split('/')[-1][:-4]+'.tif'
    driver = gdal.GetDriverByName('GTiff')
    dataset = driver.Create(ofile, width, height, 1, gdal.GDT_Int16)
    dataset.SetGeoTransform(updateGeoTransforms(geoT, xOff, yOff))
    dataset.SetProjection(proj)
    dataset.GetRasterBand(1).SetNoDataValue(-3000)
    dataset.GetRasterBand(1).WriteArray(vi_np)
    
    return ofile

In [82]:
def mosaic_images(tif_files,ofile,bbox=True):
    files_to_mosaic=[]
    for tif_file in tif_files:
        try:
            src = rasterio.open(tif_file)#;show(src, cmap='viridis')
            if bbox==True:
                print(src.bounds)
            files_to_mosaic.append(src)
        except:
            print('%s does not exist.'% tif_file)
    mosaic,out_trans=merge(files_to_mosaic)  
    out_meta=src.meta.copy()
    out_meta.update({"driver":"GTiff","height":mosaic.shape[1],"width":mosaic.shape[2],"transform":out_trans})
    !rm $ofile
    with rasterio.open(ofile,"w",**out_meta) as dest:
        dest.write(mosaic)
    return mosaic

def files_to_mosaic(hdf_files,date):
    files_to_mosaic=[]
    for hdf_file in hdf_files:
        files=hdf_file.split('.')[1]
        if str(date) in str(files):
            files_to_mosaic.append(hdf_file)
    return files_to_mosaic

def from_hdf_to_mosaic(date,hdf_files,tiffs,mosaics,bbox=True):
    files=files_to_mosaic(hdf_files,date)
    #print files
    ofiles=[hdf_to_correcttif(hdf_file,tiffs) for hdf_file in files]
    outfile=mosaics+'modis.ndvi.mosaic.'+date+'.tif'
    !rm $outfile
    mosaic=mosaic_images(ofiles,outfile,bbox=bbox)  
    return outfile

In [98]:
hdf_files=sorted(glob.glob('/Volumes/Disk 4/6month/*'))
tiffs='/Users/santhosh_ram/Desktop/Learn/MOD13Q1/tiff/'
mosaics='/Users/santhosh_ram/Desktop/Learn/MOD13Q1/mosaic/'

In [99]:
dates=pd.date_range(start='1/1/2017',end='5/31/2017', freq='16D').strftime('%y%j')
ff=['A20'+str(each) for each in dates]
print(ff)

['A2017001', 'A2017017', 'A2017033', 'A2017049', 'A2017065', 'A2017081', 'A2017097', 'A2017113', 'A2017129', 'A2017145']


In [ ]:
mosaic_files=[]
for fs in ff:
    print (fs)
    outfile=from_hdf_to_mosaic(fs,hdf_files,tiffs,mosaics,bbox=False)
    mosaic_files.append(outfile)
print(mosaic_files)

A2017001
rm: /Users/santhosh_ram/Desktop/Learn/MOD13Q1/mosaic/modis.ndvi.mosaic.A2017001.tif: No such file or directory
rm: /Users/santhosh_ram/Desktop/Learn/MOD13Q1/mosaic/modis.ndvi.mosaic.A2017001.tif: No such file or directory
A2017017
rm: /Users/santhosh_ram/Desktop/Learn/MOD13Q1/mosaic/modis.ndvi.mosaic.A2017017.tif: No such file or directory
rm: /Users/santhosh_ram/Desktop/Learn/MOD13Q1/mosaic/modis.ndvi.mosaic.A2017017.tif: No such file or directory
A2017033
rm: /Users/santhosh_ram/Desktop/Learn/MOD13Q1/mosaic/modis.ndvi.mosaic.A2017033.tif: No such file or directory
rm: /Users/santhosh_ram/Desktop/Learn/MOD13Q1/mosaic/modis.ndvi.mosaic.A2017033.tif: No such file or directory
A2017049
rm: /Users/santhosh_ram/Desktop/Learn/MOD13Q1/mosaic/modis.ndvi.mosaic.A2017049.tif: No such file or directory
rm: /Users/santhosh_ram/Desktop/Learn/MOD13Q1/mosaic/modis.ndvi.mosaic.A2017049.tif: No such file or directory
A2017065
rm: /Users/santhosh_ram/Desktop/Learn/MOD13Q1/mosaic/modis.ndvi.mos